## 6. Группировка данных и сводные таблицы

In [15]:
import pandas as pd
import seaborn as sns
import os
os.chdir('/Users/andreypovaliy/Documents/Education/ITMO_DS/05_python_for_data_analysis/')

## **GROUPBY**

Метод `groupby()` в pandas используется для группировки данных по одному или нескольким столбцам. После группировки можно применять агрегатные функции `sum(), mean(), count(), max(), min()` и др. для вычисления статистик по группам.

**1. Основной синтаксис**

```
df.groupby(by='column_name')
```

- `by='column_name'` — столбец, по которому выполняется группировка.
- Можно передавать список столбцов для многократной группировки:
```
df.groupby(['col1', 'col2'])
```

In [4]:
data = {
    'Category': ['A', 'B', 'A', 'B', 'C', 'C', 'A'],
    'Price': [100, 200, 150, 300, 400, 100, 250],
    'Quantity': [1, 2, 1, 3, 4, 2, 1]
}

df = pd.DataFrame(data)
df

,Category,Price,Quantity
0,A,100,1
1,B,200,2
2,A,150,1
3,B,300,3
4,C,400,4
5,C,100,2
6,A,250,1


**📌 1. Группировка с одной агрегатной функцией**

Допустим, нужно вычислить общее количество единиц товара (Quantity) по каждой категории (Category):

In [5]:
grouped = df.groupby('Category')['Quantity'].sum()
grouped

Category
A    3
B    5
C    6
Name: Quantity, dtype: int64

**📌 2. Группировка с несколькими агрегатными функциями**

Можно применить сразу несколько функций. Теперь мы видим сумму, среднее и максимальное значение количества проданных товаров для каждой категории.

In [6]:
grouped = df.groupby('Category')['Quantity'].agg(['sum', 'mean', 'max'])
grouped

,sum,mean,max
Category,,,
A,3,1.0,1
B,5,2.5,3
C,6,3.0,4


**📌 3. Группировка по нескольким столбцам**

Допустим, у нас есть ещё столбец Region, и мы хотим посмотреть количество проданных товаров по Category и Region.

In [7]:
df['Region'] = ['East', 'West', 'East', 'West', 'East', 'West', 'East']
grouped = df.groupby(['Category', 'Region'])['Quantity'].sum()
grouped

Category  Region
A         East      3
B         West      5
C         East      4
          West      2
Name: Quantity, dtype: int64

## Пример группировки на классическом датасете penguins

Столбцы:

	•	species — вид пингвина (Adelie, Chinstrap, Gentoo).

	•	island — остров, где был замечен пингвин (Torgersen, Biscoe, Dream).

	•	bill_length_mm, bill_depth_mm — длина и глубина клюва (мм).

	•	flipper_length_mm — длина плавника (мм).

	•	body_mass_g — масса тела (г).

	•	sex — пол (Male, Female).

In [8]:
# Загружаем датасет "penguins"
df = sns.load_dataset("penguins")

# Выведем первые 5 строк
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


**Посчитаем среднюю массу тела для каждого вида:**

In [9]:
df.groupby("species")["body_mass_g"].mean()

species
Adelie       3700.662252
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

**Найдем среднюю длину клюва (bill_length_mm) у самцов и самок разных видов:**

In [10]:
df.groupby(["species", "sex"])["bill_length_mm"].mean()

species    sex   
Adelie     Female    37.257534
           Male      40.390411
Chinstrap  Female    46.573529
           Male      51.094118
Gentoo     Female    45.563793
           Male      49.473770
Name: bill_length_mm, dtype: float64

**Допустим, мы хотим узнать среднюю и максимальную длину клюва у пингвинов разных видов:** 🐧

In [ ]:
df.groupby("species")["bill_length_mm"].agg(["mean", "max"])

,mean,max
species,,
Adelie,38.791391,46.0
Chinstrap,48.833824,58.0
Gentoo,47.504878,59.6


**Посчитаем, сколько пингвинов было замечено на каждом острове:** 🧊 🧊 🧊

In [ ]:
df.groupby("island")["species"].count()

,species
island,
Biscoe,168
Dream,124
Torgersen,52


**Допустим, мы хотим найти остров с самыми крупными пингвинами (по средней массе):**

In [ ]:
df.groupby("island")["body_mass_g"].mean().sort_values(ascending=True)

,body_mass_g
island,
Torgersen,3706.372549
Dream,3712.903226
Biscoe,4716.017964




---



## **pivot_table** в pandas

Метод `pivot_table()` используется в pandas для сводных таблиц. Он помогает агрегировать данные по категориям и представлять их в удобном виде, аналогично сводным таблицам в Excel.

```
pd.pivot_table(data, values, index, columns, aggfunc, fill_value, margins)
```
- data - DataFrame
- values - Столбец с числовыми данными, которые агрегируем
- index - Столбцы, по которым группируем строки
- columns - Столбцы, которые становятся заголовками
- aggfunc - Функция агрегирования
- fill_value - Значение, которое заполняет NaN
- margins - добавляет итоговые значения


**Теперь создадим сводную таблицу, где:**

	•	строки (index) → вид пингвина (species)

	•	колонки (columns) → остров (island)

	•	значения (values) → средняя масса тела (body_mass_g)

    •	В ячейках — средняя масса (body_mass_g) по видам и островам.

	•	NaN — значит, что на данном острове таких пингвинов не наблюдали.

In [11]:
pivot = pd.pivot_table(df, values="body_mass_g", index="species", columns="island", aggfunc="mean", fill_value=0, margins=True)
pivot

island,Biscoe,Dream,Torgersen,All
species,,,,
Adelie,3709.659091,3688.392857,3706.372549,3700.662252
Chinstrap,0.000000,3733.088235,0.000000,3733.088235
Gentoo,5076.016260,0.000000,0.000000,5076.016260
All,4716.017964,3712.903226,3706.372549,4201.754386


**Можно использовать несколько функций сразу:**

In [12]:
pivot = pd.pivot_table(df, values="body_mass_g", index="species", columns="island", aggfunc=["mean", "max", "count"])
pivot

mean                               max                    \
island          Biscoe        Dream    Torgersen  Biscoe   Dream Torgersen   
species                                                                      
Adelie     3709.659091  3688.392857  3706.372549  4775.0  4650.0    4700.0   
Chinstrap          NaN  3733.088235          NaN     NaN  4800.0       NaN   
Gentoo     5076.016260          NaN          NaN  6300.0     NaN       NaN   

           count                  
island    Biscoe Dream Torgersen  
species                           
Adelie      44.0  56.0      51.0  
Chinstrap    NaN  68.0       NaN  
Gentoo     123.0   NaN       NaN

**Можно агрегировать несколько столбцов:**

In [ ]:
pivot = pd.pivot_table(df, values=["body_mass_g", "flipper_length_mm"], index="species", aggfunc="mean")
pivot

,body_mass_g,flipper_length_mm
species,,
Adelie,3700.662252,189.953642
Chinstrap,3733.088235,195.823529
Gentoo,5076.016260,217.186992


**Флаг margins=True добавляет итоговую строку и колонку:**

	•	Строка All показывает общую среднюю массу по всем видам.
	•	Колонка All показывает общую среднюю массу по островам.

In [ ]:
pivot = pd.pivot_table(df, values="body_mass_g", index="species", columns="island", aggfunc="mean", margins=True)
pivot

island,Biscoe,Dream,Torgersen,All
species,,,,
Adelie,3709.659091,3688.392857,3706.372549,3700.662252
Chinstrap,NaN,3733.088235,NaN,3733.088235
Gentoo,5076.016260,NaN,NaN,5076.016260
All,4716.017964,3712.903226,3706.372549,4201.754386


# Работа с реальным примером



In [16]:
df = pd.read_excel('./lect/data/T_cashback_dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59826 entries, 0 to 59825
Data columns (total 57 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ключ_клиента                      59826 non-null  int64         
 1   регион_проживания                 59826 non-null  object        
 2   город_проживания                  59826 non-null  object        
 3   возраст                           59826 non-null  int64         
 4   пол                               59724 non-null  object        
 5   месяц_покупок                     59826 non-null  datetime64[ns]
 6   оборот_аптеки                     44780 non-null  float64       
 7   оборот_рестораны                  46427 non-null  float64       
 8   оборот_одежда_и_обувь             37356 non-null  float64       
 9   оборот_автоуслуги                 19035 non-null  float64       
 10  оборот_супермаркеты               55462 non-nu

In [17]:
df.head()

,ключ_клиента,регион_проживания,город_проживания,возраст,пол,месяц_покупок,оборот_аптеки,оборот_рестораны,оборот_одежда_и_обувь,оборот_автоуслуги,...,кэшбэк_развлечения,кэшбэк_жд_билеты,кэшбэк_образование,кэшбэк_дом_и_ремонт,кэшбэк_спорттовары,кэшбэк_животные,кэшбэк_цветы,кэшбэк_фастфуд,кэшбэк_каршеринг,кэшбэк_аренда_авто
0,1889,Москва,Москва,50,M,2023-04-01,18993.0,99900.0,3294.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1889,Москва,Москва,50,M,2023-05-01,37807.0,92424.0,NaN,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1889,Москва,Москва,50,M,2023-06-01,20791.0,124800.0,51246.0,67490.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1889,Москва,Москва,50,M,2023-07-01,131129.0,18060.0,78120.0,2400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1889,Москва,Москва,50,M,2023-08-01,44096.0,198480.0,19080.0,147705.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.columns

Index(['ключ_клиента', 'регион_проживания', 'город_проживания', 'возраст',
       'пол', 'месяц_покупок', 'оборот_аптеки', 'оборот_рестораны',
       'оборот_одежда_и_обувь', 'оборот_автоуслуги', 'оборот_супермаркеты',
       'оборот_такси', 'оборот_красота', 'оборот_развлечения',
       'оборот_жд_билеты', 'оборот_образование', 'оборот_дом_и_ремонт',
       'оборот_спорттовары', 'оборот_животные', 'оборот_цветы',
       'оборот_фастфуд', 'оборот_каршеринг', 'оборот_аренда_авто',
       'активация_кэшбэка_аптеки', 'активация_кэшбэка_рестораны',
       'активация_кэшбэка_одежда_и_обувь', 'активация_кэшбэка_автоуслуги',
       'активация_кэшбэка_супермаркеты', 'активация_кэшбэка_такси',
       'активация_кэшбэка_красота', 'активация_кэшбэка_развлечения',
       'активация_кэшбэка_жд_билеты', 'активация_кэшбэка_образование',
       'активация_кэшбэка_дом_и_ремонт', 'активация_кэшбэка_спорттовары',
       'активация_кэшбэка_животные', 'активация_кэшбэка_цветы',
       'активация_кэшбэка_фа

In [19]:
df1 = df[['ключ_клиента', 'регион_проживания', 'город_проживания', 'возраст',
       'пол', 'месяц_покупок', 'оборот_аптеки', 'оборот_рестораны',
       'оборот_одежда_и_обувь', 'оборот_автоуслуги', 'оборот_супермаркеты',
       'оборот_такси', 'оборот_красота', 'оборот_развлечения',
       'оборот_жд_билеты', 'оборот_образование', 'оборот_дом_и_ремонт',
       'оборот_спорттовары', 'оборот_животные', 'оборот_цветы',
       'оборот_фастфуд', 'оборот_каршеринг', 'оборот_аренда_авто',]]

In [20]:
df1.shape

(59826, 23)

In [22]:
df2 = df[['ключ_клиента', 'регион_проживания', 'город_проживания', 'возраст',
       'пол', 'месяц_покупок', 'активация_кэшбэка_аптеки', 'активация_кэшбэка_рестораны',
       'активация_кэшбэка_одежда_и_обувь', 'активация_кэшбэка_автоуслуги',
       'активация_кэшбэка_супермаркеты', 'активация_кэшбэка_такси',
       'активация_кэшбэка_красота', 'активация_кэшбэка_развлечения',
       'активация_кэшбэка_жд_билеты', 'активация_кэшбэка_образование',
       'активация_кэшбэка_дом_и_ремонт', 'активация_кэшбэка_спорттовары',
       'активация_кэшбэка_животные', 'активация_кэшбэка_цветы',
       'активация_кэшбэка_фастфуд', 'активация_кэшбэка_каршеринг',
       'активация_кэшбэка_аренда_авто',]]

In [23]:
df2.shape

(59826, 23)

In [ ]:
df3 = df[['ключ_клиента', 'регион_проживания', 'город_проживания', 'возраст',
       'пол', 'месяц_покупок', 'кэшбэк_аптеки', 'кэшбэк_рестораны',
       'кэшбэк_одежда_и_обувь', 'кэшбэк_автоуслуги', 'кэшбэк_супермаркеты',
       'кэшбэк_такси', 'кэшбэк_красота', 'кэшбэк_развлечения',
       'кэшбэк_жд_билеты', 'кэшбэк_образование', 'кэшбэк_дом_и_ремонт',
       'кэшбэк_спорттовары', 'кэшбэк_животные', 'кэшбэк_цветы',
       'кэшбэк_фастфуд', 'кэшбэк_каршеринг', 'кэшбэк_аренда_авто']]

In [ ]:
df3.shape

(59826, 23)

In [ ]:
id_vars = ['ключ_клиента', 'регион_проживания', 'город_проживания', 'возраст', 'пол', 'месяц_покупок']

In [ ]:
df1 = pd.melt(df1, id_vars=id_vars, var_name='cashback_category', value_name='spent')
df2 = pd.melt(df2, id_vars=id_vars, var_name='cashback_category', value_name='is_active')
df3 = pd.melt(df3, id_vars=id_vars, var_name='cashback_category', value_name='cashback')


In [ ]:
df4 = pd.concat([df1, df2['is_active'], df3['cashback']], axis=1)
df4

,ключ_клиента,регион_проживания,город_проживания,возраст,пол,месяц_покупок,cashback_category,spent,is_active,cashback
0,1889,Москва,Москва,50,M,2023-04-01,оборот_аптеки,18993.0,1.0,864.0
1,1889,Москва,Москва,50,M,2023-05-01,оборот_аптеки,37807.0,1.0,1884.0
2,1889,Москва,Москва,50,M,2023-06-01,оборот_аптеки,20791.0,1.0,21.0
3,1889,Москва,Москва,50,M,2023-07-01,оборот_аптеки,131129.0,1.0,6033.0
4,1889,Москва,Москва,50,M,2023-08-01,оборот_аптеки,44096.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1017037,1005319581,Москва,Москва,37,NaN,2023-05-01,оборот_аренда_авто,NaN,NaN,NaN
1017038,1005319581,Москва,Москва,37,NaN,2023-06-01,оборот_аренда_авто,NaN,NaN,NaN
1017039,1005319581,Москва,Москва,37,NaN,2023-07-01,оборот_аренда_авто,NaN,NaN,NaN
1017040,1005319581,Москва,Москва,37,NaN,2023-08-01,оборот_аренда_авто,NaN,NaN,NaN


In [ ]:
df4 = df4.rename(columns={'ключ_клиента': 'client_id',
                          'регион_проживания': 'region',
                          'город_проживания' : 'city',
                          'возраст': 'age',
                          'пол' : 'gender',
                          'месяц_покупок' : 'month'})
df4

,client_id,region,city,age,gender,month,cashback_category,spent,is_active,cashback
0,1889,Москва,Москва,50,M,2023-04-01,оборот_аптеки,18993.0,1.0,864.0
1,1889,Москва,Москва,50,M,2023-05-01,оборот_аптеки,37807.0,1.0,1884.0
2,1889,Москва,Москва,50,M,2023-06-01,оборот_аптеки,20791.0,1.0,21.0
3,1889,Москва,Москва,50,M,2023-07-01,оборот_аптеки,131129.0,1.0,6033.0
4,1889,Москва,Москва,50,M,2023-08-01,оборот_аптеки,44096.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1017037,1005319581,Москва,Москва,37,NaN,2023-05-01,оборот_аренда_авто,NaN,NaN,NaN
1017038,1005319581,Москва,Москва,37,NaN,2023-06-01,оборот_аренда_авто,NaN,NaN,NaN
1017039,1005319581,Москва,Москва,37,NaN,2023-07-01,оборот_аренда_авто,NaN,NaN,NaN
1017040,1005319581,Москва,Москва,37,NaN,2023-08-01,оборот_аренда_авто,NaN,NaN,NaN


In [ ]:
df4['cashback_category'] = df4['cashback_category'].str.replace("оборот_", "").str.replace("_", " ")
df4

,client_id,region,city,age,gender,month,cashback_category,spent,is_active,cashback
0,1889,Москва,Москва,50,M,2023-04-01,аптеки,18993.0,1.0,864.0
1,1889,Москва,Москва,50,M,2023-05-01,аптеки,37807.0,1.0,1884.0
2,1889,Москва,Москва,50,M,2023-06-01,аптеки,20791.0,1.0,21.0
3,1889,Москва,Москва,50,M,2023-07-01,аптеки,131129.0,1.0,6033.0
4,1889,Москва,Москва,50,M,2023-08-01,аптеки,44096.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1017037,1005319581,Москва,Москва,37,NaN,2023-05-01,аренда авто,NaN,NaN,NaN
1017038,1005319581,Москва,Москва,37,NaN,2023-06-01,аренда авто,NaN,NaN,NaN
1017039,1005319581,Москва,Москва,37,NaN,2023-07-01,аренда авто,NaN,NaN,NaN
1017040,1005319581,Москва,Москва,37,NaN,2023-08-01,аренда авто,NaN,NaN,NaN


In [ ]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017042 entries, 0 to 1017041
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   client_id          1017042 non-null  int64         
 1   region             1017042 non-null  object        
 2   city               1017042 non-null  object        
 3   age                1017042 non-null  int64         
 4   gender             1015308 non-null  object        
 5   month              1017042 non-null  datetime64[ns]
 6   cashback_category  1017042 non-null  object        
 7   spent              448181 non-null   float64       
 8   is_active          229758 non-null   float64       
 9   cashback           93384 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 77.6+ MB


In [ ]:
df4 = df4.dropna(subset=['gender'])
df4 = df4.fillna(0)



In [ ]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1015308 entries, 0 to 1017035
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   client_id          1015308 non-null  int64         
 1   region             1015308 non-null  object        
 2   city               1015308 non-null  object        
 3   age                1015308 non-null  int64         
 4   gender             1015308 non-null  object        
 5   month              1015308 non-null  datetime64[ns]
 6   cashback_category  1015308 non-null  object        
 7   spent              1015308 non-null  float64       
 8   is_active          1015308 non-null  float64       
 9   cashback           1015308 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 85.2+ MB


In [ ]:
df4["is_active"] = df4["is_active"].astype('int64')
df4

,client_id,region,city,age,gender,month,cashback_category,spent,is_active,cashback
0,1889,Москва,Москва,50,M,2023-04-01,аптеки,18993.0,1,864.0
1,1889,Москва,Москва,50,M,2023-05-01,аптеки,37807.0,1,1884.0
2,1889,Москва,Москва,50,M,2023-06-01,аптеки,20791.0,1,21.0
3,1889,Москва,Москва,50,M,2023-07-01,аптеки,131129.0,1,6033.0
4,1889,Москва,Москва,50,M,2023-08-01,аптеки,44096.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
1017031,1003493065,Республика Карелия,Петрозаводск,42,M,2023-05-01,аренда авто,0.0,0,0.0
1017032,1003493065,Республика Карелия,Петрозаводск,42,M,2023-06-01,аренда авто,0.0,0,0.0
1017033,1003493065,Республика Карелия,Петрозаводск,42,M,2023-07-01,аренда авто,0.0,0,0.0
1017034,1003493065,Республика Карелия,Петрозаводск,42,M,2023-08-01,аренда авто,0.0,0,0.0


Задачи найти:

- общие суммы кешбека, выплаченные клиентам банка по категориям и месяцам.

- средний кешбек в категории по каждой категории по месяцам

- максимальный кешбек в каждой категории по месяцам

In [ ]:
grouped = df4.groupby(["month", "cashback_category"])["cashback"].sum().reset_index()
grouped

,month,cashback_category,cashback
0,2023-04-01,автоуслуги,3865298.0
1,2023-04-01,аптеки,1884250.0
2,2023-04-01,аренда авто,0.0
3,2023-04-01,дом и ремонт,1021887.0
4,2023-04-01,жд билеты,947952.0
...,...,...,...
97,2023-09-01,спорттовары,1137660.0
98,2023-09-01,супермаркеты,8914728.0
99,2023-09-01,такси,2703002.0
100,2023-09-01,фастфуд,0.0


In [ ]:
month_filter = "2023-05-01"
groped_filtered = grouped[grouped["month"] == month_filter]
groped_filtered.sort_values(by="cashback", ascending=False)

,month,cashback_category,cashback
26,2023-05-01,одежда и обувь,8275516.0
28,2023-05-01,рестораны,5259030.0
17,2023-05-01,автоуслуги,4840320.0
30,2023-05-01,супермаркеты,4677257.0
18,2023-05-01,аптеки,2808736.0
31,2023-05-01,такси,2222975.0
27,2023-05-01,развлечения,2130829.0
24,2023-05-01,красота,2047804.0
21,2023-05-01,жд билеты,1489936.0
20,2023-05-01,дом и ремонт,1184319.0


In [24]:
df_pivot = df4.pivot_table(
    index="cashback_category",
    columns="month",
    values="cashback",
    aggfunc=["mean"],
    margins=True
)

df_pivot.round(2)

NameError: name 'df4' is not defined

In [ ]:
print(df_pivot.columns)

MultiIndex([('mean', 2023-04-01 00:00:00),
            ('mean', 2023-05-01 00:00:00),
            ('mean', 2023-06-01 00:00:00),
            ('mean', 2023-07-01 00:00:00),
            ('mean', 2023-08-01 00:00:00),
            ('mean', 2023-09-01 00:00:00),
            ('mean',               'All')],
           names=[None, 'month'])


In [ ]:
df_pivot.columns = ['2023-04-01','2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01']

In [ ]:
from datetime import datetime
df_pivot.columns = [datetime(2023, 4, 1),
                    datetime(2023, 5, 1),
                    datetime(2023, 6, 1),
                    datetime(2023, 7, 1),
                    datetime(2023, 8, 1),
                    datetime(2023, 9, 1)]

In [ ]:
df_pivot